# Welding Defect Image Classification 

Input dataset is a set of welded metal images which includes defective and non-defective ones. Classification is implemented considering three classes or target variables - cluster_porosity , cracks and no_defect.

## Quick check on image distribution

In [4]:
%%bash
ls weld_images/cluster_porosity | wc -l
ls weld_images/cracks | wc -l
ls weld_images/no_defect | wc -l

8
2
10


## Import required packages

In [7]:
import pandas as pd
import numpy as np
import random
import os
import pickle
import shutil
import cv2

import keras
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Sequential
from sklearn.metrics import classification_report
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

## Initialize parameters

In [8]:
EPOCHS = 30
INIT_LR = 1e-3
BS = 8
IMAGE_DIMS = (128,800, 3)

## Specify classes or target variables

In [9]:
#directory = 'E:\\Welding defects classification\\Use Case 1-Welding Images'
directory = 'weld_images'
categories = ['cluster_porosity','cracks','no_defect']

## Perform data augmentation

The available dataset is very small because of which training may not produce good results. Hence, we perform data augmentation on the available dataset.

In [28]:
# Function for carrying out data augmentation on the existing dataset

def perform_data_aug(root_dir, category_list, aug_limit_list):

        # Initialise ImageDataGenerator class.
        datagen = ImageDataGenerator(
                      rotation_range = 40,
                      shear_range = 0.2,
                      zoom_range = 0.2,
                      horizontal_flip = True,
                      vertical_flip = True,
                      brightness_range = (0.5, 1.5))
        
        for folder,limit in zip(category_list,aug_limit_list):
            p = os.path.join(root_dir, folder)
            paths = list(os.walk(p))
            
            # Create directory for augmented images for every class
            if not os.path.exists(folder):
                     os.makedirs('preview_'+ folder)
                    
            for f in paths[0][2]:
                
                #Load sample image
                full_path = os.path.join(p,f)
                img = load_img(full_path)
                
                # Converting the input sample image to an array
                x = img_to_array(img)
                
                # Reshaping the input image
                x = x.reshape((1, ) + x.shape)

            # Generate and save required no. of augmented samples
            i = 0
            for batch in datagen.flow(x, batch_size = 1,
                            save_to_dir ='preview_' + folder,
                            save_prefix ='image_aug', save_format ='jpg'):
                 i += 1
                 if i > limit:
                    break

In [29]:
# Call data augmentation function with the list of augmented images required for each class
# This step also solves class imbalance issue which is seen in the dataset

aug_limit_list = [70,80,70]
perform_data_aug(directory, categories, aug_limit_list)

In [30]:
# Copy augmented images into respective class folders
for c in categories:
    src = 'preview_' + c
    dest = os.path.join(directory, c)
    # Fetch all files
    for file_name in os.listdir(src):
        # Construct full file path
        source = os.path.join(src,file_name)
        # Move only files
        if os.path.isfile(source) and source.endswith('jpg'):
            shutil.copy(source, dest)
            #print('Moved:', file_name)

    # Delete empty augment images directory
    shutil.rmtree(src)

## Resize images and tag them 

In [37]:
data = []
for category in categories:
    path = os.path.join(directory, category)
    for img in os.listdir(path):
        #print(img)
        img_path = os.path.join(path, img)
        label = category
        arr = cv2.imread(img_path)
        new_arr = cv2.resize(arr, (IMAGE_DIMS[1],IMAGE_DIMS[0]))
        data.append([new_arr,label])

In [38]:
len(data)

242

## Separate data into features and labels

In [39]:
random.seed(42)
random.shuffle(data)

X, y = [],[]
for features,labels in data:
    X.append(features)
    y.append(labels)

## Perform normalisation on features

In [40]:
X = np.array(X, dtype="float") / 255.0
y = np.array(y)
print("[INFO] data matrix: {:.2f}MB".format(X.nbytes / (1024 * 1000.0)))

[INFO] data matrix: 580.80MB


## Binarize the labels or classes

In [41]:
lb = LabelBinarizer()
y = lb.fit_transform(y)

## Split train and test data

In [42]:
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
print(trainX.shape, trainX.dtype)
print(testX.shape, testX.dtype)
print(trainY.shape, trainY.dtype)
print(testY.shape, testY.dtype)

(193, 128, 800, 3) float64
(49, 128, 800, 3) float64
(193, 3) int64
(49, 3) int64


In [ ]:
# # Account for skew in the labeled data
# classTotals = trainY.sum(axis=0)
# classWeight = classTotals.max() / classTotals

# category_ind = [categories.index(c) for c in categories]
# class_weight_dic = dict(zip(category_ind, classWeight.tolist()))
# class_weight_dic

In [44]:
# Construct image generator for data augmentation on the fly
aug = ImageDataGenerator(rotation_range=False, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, vertical_flip=True, fill_mode="nearest")

## Build the model

In [45]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model


## Initialize model with parameters

In [46]:
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
                            depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Compile the model

In [47]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## Create checkpoints

In [48]:
filepath="my_checkpoints/epochs:{epoch:03d}-val_acc:{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor= 'val_accuracy' , verbose=1, save_best_only=True,
    mode= 'max' )
callbacks_list = [checkpoint]

## Train model

In [49]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(
       aug.flow(trainX, trainY, batch_size=BS),
       validation_data=(testX, testY),
       steps_per_epoch=len(trainX) // BS,
       epochs=EPOCHS,
       # class_weight=class_weight_dic, 
       callbacks=callbacks_list,
       verbose=1)

[INFO] training network...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/30
24/24 [==============================] - ETA: 0s - loss: 0.9172 - accuracy: 0.8757
Epoch 1: val_accuracy improved from -inf to 0.22449, saving model to my_checkpoints/epochs:001-val_acc:0.224.hdf5
24/24 [==============================] - 102s 4s/step - loss: 0.9172 - accuracy: 0.8757 - val_loss: 5.5829 - val_accuracy: 0.2245
Epoch 2/30
24/24 [==============================] - ETA: 0s - loss: 0.6527 - accuracy: 0.8000
Epoch 2: val_accuracy did not improve from 0.22449
24/24 [==============================] - 89s 4s/step - loss: 0.6527 - accuracy: 0.8000 - val_loss: 7.2033 - val_accuracy: 0.2245
Epoch 3/30
24/24 [==============================] - ETA: 0s - loss: 0.5877 - accuracy: 0.8216
Epoch 3: val_accuracy did not improve from 0.22449
24/24 [==============================] - 89s 4s/step - loss: 0.5877 - accuracy: 0.8216 - val_loss: 8.3854 - val_accuracy: 0.2245
Epoch 4/30
24/24 [==============================] - ETA: 0s - loss: 0.4175 - accuracy: 0.8757
Epoch 4: val_accurac

## Save model

In [50]:
# save the model to disk
print("[INFO] serializing network...")
model.save('results/weld_3cls.model')

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open('results/lb.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network...
INFO:tensorflow:Assets written to: results/weld_3cls.model/assets
[INFO] serializing label binarizer...


In [51]:
# Save training history
# Convert history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(H.history)
hist_csv_file = 'results/history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

## Predict on test data

In [52]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] evaluating network...


In [53]:
# Retrieve index of the label corresponding largest predicted probability for each image in testing set using argmax
predIdxs = np.argmax(predIdxs, axis=1)
predIdxs

array([1, 2, 2, 0, 0, 2, 1, 1, 0, 2, 2, 1, 1, 0, 2, 1, 0, 1, 2, 1, 2, 2,
       2, 2, 1, 2, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 2, 1, 0, 0, 0, 0, 1, 0,
       2, 1, 2, 2, 2])

## Evaluate model

In [54]:
score = model.evaluate(testX, testY, verbose = 1)

2/2 [==============================] - 6s 2s/step - loss: 0.1813 - accuracy: 0.9592


In [57]:
loss, accuracy = score[0], score[1]
print("The loss is found to be %.2f and accuracy is found to be %.2f percent !" %(loss,accuracy * 100))

The loss is found to be 0.18 and accuracy is found to be 95.92 percent !


## Display classification report

In [59]:
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

                  precision    recall  f1-score   support

cluster_porosity       0.91      0.91      0.91        11
          cracks       1.00      1.00      1.00        17
       no_defect       0.95      0.95      0.95        21

        accuracy                           0.96        49
       macro avg       0.95      0.95      0.95        49
    weighted avg       0.96      0.96      0.96        49

